In [1]:
from google.colab import drive
drive.mount('/content/drive/')

MessageError: ignored

In [2]:
import cv2
import numpy as np
import glob
from skimage import io

width=100
height=100


Data=[]
Target=[]
Test=[]
Target1=[]



# genuies signature
file_list = glob.glob(r'/content/drive/My Drive/DataSet/Signature/dataset/genuines/*.png')
for i in file_list:
  im=cv2.imread(i)
  im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
  im=cv2.resize(im,(width,height))
  Data.append(im)
  Target.append(0)


  # forgeries signature
file_list1 = glob.glob('/content/drive/My Drive/DataSet/Signature/dataset/forgeries/*.png')

for i in file_list:
  im=cv2.imread(i)
  im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
  im=cv2.resize(im,(width,height))
  Data.append(im)
  Target.append(1)

# forgeries signature Test
file_list1 = glob.glob(r'/content/drive/My Drive/DataSet/Signature/sample_Signature/forged/*.PNG')

for i in file_list1:
  im=cv2.imread(i)
  im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
  im=cv2.resize(im,(width,height))
  Test.append(im)
  Target1.append(0)

file_list = glob.glob(r'/content/drive/My Drive/DataSet/Signature/sample_Signature/genuine/*.PNG')
# genuies signature
for i in file_list:
  im=cv2.imread(i)
  im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
  im=cv2.resize(im,(width,height))
  Test.append(im)
  Target1.append(1)

# genuies signature
file_list1 = glob.glob(r'/content/drive/My Drive/DataSet/Signature/sample_Signature/genuine/*.png')
for i in file_list1:
  im=cv2.imread(i)
  im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
  im=cv2.resize(im,(width,height))
  Test.append(im)
  Target1.append(0)

  # forgeries signature
file_list1 = glob.glob(r'/content/drive/My Drive/DataSet/Signature/sample_Signature/forged/*.png')

for i in file_list1:
  im=cv2.imread(i)
  im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
  im=cv2.resize(im,(width,height))
  Test.append(im)
  Target1.append(1)



In [3]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(Target)

In [4]:
from tensorflow.keras.utils import to_categorical
y_test = to_categorical(Target1)

In [5]:
Final_Data=np.reshape(Data,(len(Data),height,width,1))

In [6]:
Final_Test=np.reshape(Test,(len(Test),height,width,1))

In [7]:
# CNN LSTM
from keras import layers
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv2D,SeparableConv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import TimeDistributed
from keras.layers import LSTM
from keras.models import Model

myInput=layers.Input(shape=(height,width,1))
conv1=SeparableConv2D(64,3, activation='relu',padding='same')(myInput)
max1=MaxPooling2D(2)(conv1)
conv2=SeparableConv2D(64,3, activation='relu',padding='same')(max1)
max2=MaxPooling2D(2)(conv2)
Time1=layers.TimeDistributed(Flatten())(max2)
lstm1=LSTM(150)(Time1)
des1 =Dense(50, activation='relu')(lstm1)
Extract_Model = Model(myInput, des1)
des2 =Dense(20, activation='relu')(des1)
out_layer =Dense(2, activation='softmax')(des2)
model = Model(myInput, out_layer)
model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

verbose, epochs, batch_size = 1, 10, 32
model.fit(Final_Data,y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 1)]     0         
                                                                 
 separable_conv2d (Separable  (None, 100, 100, 64)     137       
 Conv2D)                                                         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 50, 50, 64)       0         
 )                                                               
                                                                 
 separable_conv2d_1 (Separab  (None, 50, 50, 64)       4736      
 leConv2D)                                                       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 25, 25, 64)       0         
 2D)                                                       

In [8]:
trainFeature=(Extract_Model.predict(Final_Data))
testFeature=(Extract_Model.predict(Final_Test))

10/10 [==============================] - 4s 362ms/step


In [9]:
from sklearn import svm
classifier_svm = svm.LinearSVC()
classifier_svm.fit(trainFeature,Target)

LinearSVC()

In [10]:
from sklearn import tree
from sklearn.metrics import classification_report, accuracy_score
classifier_tree = tree.DecisionTreeClassifier()
classifier_tree.fit(trainFeature,Target)



DecisionTreeClassifier()

In [11]:
from sklearn.neighbors import KNeighborsClassifier
classifier_Knn = KNeighborsClassifier(n_neighbors=3)
classifier_Knn.fit(trainFeature,Target)

KNeighborsClassifier(n_neighbors=3)

In [14]:
from sklearn.ensemble import  VotingClassifier
classifier_Voting = VotingClassifier(estimators=[('SVM', classifier_svm), ('TREE', classifier_tree), ('KNN', classifier_Knn)],weights=[1,1,1])
classifier_Voting.fit(trainFeature,Target)
Class_Voting=classifier_Voting.predict(testFeature)

In [15]:
from sklearn.metrics import classification_report, accuracy_score
print(classification_report(Target1,Class_Voting))

              precision    recall  f1-score   support

           0       0.60      0.99      0.75       180
           1       0.33      0.01      0.02       120

    accuracy                           0.60       300
   macro avg       0.47      0.50      0.38       300
weighted avg       0.49      0.60      0.45       300



In [16]:
print('Accuracy= ' + str(round(accuracy_score(Target1, Class_Voting)*100)))

Accuracy= 60
